In [1]:
import os
import openai
import time
import random

In [ ]:
openai.api_key = "" # Use your own API Key here

In [6]:
model_name = "text-davinci-003"
rate_limit_per_minute = 40000 # 150000 # 1000 requests per minute
last_rate_check_time = None
rate_per_minute = 0
max_total_token_cnt = 3000 # 3000 tokens per request
def query_codex(prompt, n = 1, temperature = 0.8, max_tokens = 250, max_retries = 4, top_p = 0.75, print_prompts=False):
    res = None
    try_number = 0
    global last_rate_check_time, rate_per_minute
    max_tokens = max(max_tokens // 5, 56)
    total_token_count = len(prompt)/4 + max_tokens # 4 tokens per char
    if total_token_count >= max_total_token_cnt:
        print(f"Too big request: {total_token_count} tokens!!")
        yield from [] # return nothing if the request is too big
        return
    time_now = time.time() # time in seconds since epoch
    temp_rate_per_minute = rate_per_minute
    if last_rate_check_time is None:
        last_rate_check_time = time_now
    if time_now - last_rate_check_time > 60:
        last_rate_check_time = time_now
        temp_rate_per_minute = rate_per_minute * 0.98 + total_token_count
    else:
        temp_rate_per_minute += total_token_count
    pred_num_req_per_minute = rate_per_minute
    # Simple Rate limiter
    while pred_num_req_per_minute > rate_limit_per_minute * 0.85:
        time_now = time.time() # time in seconds since epoch
        pred_num_req_per_minute = temp_rate_per_minute/(time_now - last_rate_check_time)
        print(f"Premptive sleeping to reduce rate limit: {pred_num_req_per_minute} requests per minute")
        time.sleep(1) # sleep for 1 second
    if temp_rate_per_minute > rate_limit_per_minute * 0.85:
        temp_rate_per_minute = rate_limit_per_minute * 0.42 # half the rate limit
    if time_now - last_rate_check_time > 60:
        last_rate_check_time = time_now
    while True:
        try:
            print("Sending request...")
            print(f"Request made: {rate_per_minute} requests per minute")
            print(f"Last token count: {total_token_count}, token_request = {max_tokens}")
            start = time.time()
            # print("=======================================================")
            # print(f"{prompt}")           
            res = openai.Completion.create(
                model = model_name,
                prompt = prompt,
                max_tokens = max_tokens,
                temperature = temperature,
                n = n,
                top_p = top_p)
            end = time.time()
            # print("==========================================================")
            print(f"Response received!! time taken: {end - start}")
            print(f"Request made: {rate_per_minute} requests per minute")
            print(f"Last token count: {total_token_count}")
            print(f"Total token acc to OpenAI: {res['usage']['total_tokens']}")
            rate_per_minute = 0.98 * rate_per_minute + res['usage']['total_tokens']
            break
        except (openai.error.RateLimitError, openai.error.APIError, openai.error.InvalidRequestError) as e:
            if isinstance(e, openai.error.InvalidRequestError):
                print(f"A big request was made. Skipping... {e}")
                yield from []
                return
            else:
                if try_number == max_retries:
                    print("Reached the max rate limit error: Giving up!")
                    yield from []
                    return
                sleep_secs = 10 * (2 ** try_number) # Exponential retry
                try_number += 1
                time_now = time.time() # time in seconds since epoch
                pred_num_req_per_minute = rate_per_minute/(time_now - last_rate_check_time)
                last_rate_check_time = time.time()
                print(e)
                print(f"Rate limit error #{try_number}: Sleeping for {sleep_secs} seconds...")
                print(f"Current rate per minute: {pred_num_req_per_minute}, earlier rate per minute: {rate_per_minute}")
                time.sleep(sleep_secs)
    for ch in res["choices"]:
        #print("===================================PROMPT START===================================================================================")           
        #print(ch["text"])
        #print(print_prompts, "===================================PROMPT END===================================================================================")
        yield ch["text"]


In [20]:
prompt = "Ask Alexa in fun ways to quickly summarize the events on the calender today"

In [21]:
for res in list(query_codex(prompt)):
    print(res)

Sending request...
Request made: 353.56733881599996 requests per minute
Last token count: 74.75, token_request = 56
Response received!! time taken: 1.1920568943023682
Request made: 353.56733881599996 requests per minute
Last token count: 74.75
Total token acc to OpenAI: 59


"Today you have a busy day ahead! You have a meeting at 9am, lunch at 12pm, and a conference call at 3pm. Don't forget to take a break in between to recharge!"
